# TrustyAI
By testing our model after training and before it goes into production, we get a good understand on how well the model performs on data which we currently have available.  
Unfortunatly, the real world is not very simple and things are constantly changing, so what we took for granted yesterday may look completely different today. Because of this, we need to monitor if changes in the world has an impact on our models ability to make accurate predictions.  
We want to track key metrics that can indicate if the model or the data starts behaving strangely once the model is in production and sees live data.  
For this, we have a tool called TrustyAI that can help us generate metrics for fairness and drift detection. With TrustyAI, we can determine if live data differs from what we expected when we created our training dataset. It allows us to measure whether our model responds to real-world data differently than we initially anticipated.

In [ ]:
!pip -q install onnxruntime model-registry==0.2.10

### Get your User token
We need to provide our user token so that the workbench can send request to our TrustyAI Service.
You can get the user token by:
1. Going to the OpenShift Console
2. Click the dropdown in the top right corner where your username is displayed
3. Choose "Copy login command"
4. Log in
5. Select the part in the first code box after `token`, it should look something like `sha256~.....`

In [ ]:
infer_endpoint = "ENTER-YOUR-INFERENCE-ENDPOINT"
token = "ENTER-YOUR-TOKEN"
model_version = "ENTER-YOUR-MODEL-VERSION"
cluster_domain = "ENTER-YOUR-CLUSTER-DOMAIN"

model_name = "jukebox"

In [ ]:
namespace_file_path =\
    '/var/run/secrets/kubernetes.io/serviceaccount/namespace'
with open(namespace_file_path, 'r') as namespace_file:
    current_namespace = namespace_file.read()

In [4]:
import pandas as pd
import pickle
import json
import onnxruntime as rt
import numpy as np
import onnx

import requests
from urllib.parse import urljoin

### Data
We are going to send our training data to the TrustyAI Service so that it can compare the training data with the new data coming in from our inference requests.  
We limit it to just 5000 samples to keep it light, but in a real usecase you would send your full training data, or a part of it that properly represented its distribution.

In [ ]:
X_train = pd.read_pickle('train_data.pkl')[0][:5000]

with open("scaler.pkl", 'rb') as handle:
    scaler = pickle.load(handle)
data = scaler.transform(X_train)

We also get the predictions of the data so that we can see if they start drifting as well.

In [ ]:
sess = rt.InferenceSession("onnx_model.onnx", providers=rt.get_available_providers())
input_name = sess.get_inputs()[0].name
output_name = sess.get_outputs()[0].name
y_pred_temp = sess.run([output_name], {input_name: data.astype(np.float32).tolist()})

After we have all our data, we structure it in a specific way that TrustyAI expects.  
Noteably, we add a data_tag to our data, so that we can keep track of different iterations of it or data that's used for different purposes.

In [10]:
training_data = {
    "model_name": model_name,
    "data_tag": "TRAINING",
    "request": {
        "inputs": [
            {
                "name": input_name,
                "shape": np.shape(data),
                "datatype": "FP32",
                "data": data.tolist()
            }
        ]    
    },
    "response": {
        "model_name": model_name,
        "model_version": "1",
        "outputs": [
            {
                "name": output_name,
                "datatype": "FP32",
                "shape": np.shape(y_pred_temp[0]),
                "data": y_pred_temp[0].tolist()
            }
        ]
    }
}


### Interacting with TrustyAI through requests
We will be interacting with our TrustyAI Service through rest requests.  
Before we can do that though, we need to add our TrustyAI Service Route as the `base_url` so we know where to send the requests.

In [11]:
base_url = f"http://trustyai-service.{current_namespace}-test.svc.cluster.local"
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

First thing we do is upload the data which we have prepared.

In [12]:
# Upload data
endpoint = "data/upload"
url = urljoin(base_url, endpoint)
response = requests.post(url, headers=headers, json=training_data)
print(response.text)

5000 datapoints successfully added to jukebox data.


Then we change the names of the inputs. By default they will just be the input-layer name with a number behind them, but in our case we can change them to the actual feature names.

In [ ]:
# Change the input and output names
endpoint = "info/names"
url = urljoin(base_url, endpoint)

payload = {
    "modelId": model_name,
    "inputMapping": {
        f"{input_name}-0": "is_explicit",
        f"{input_name}-1": "duration_ms",
        f"{input_name}-2": "danceability",
        f"{input_name}-3": "energy",
        f"{input_name}-4": "key",
        f"{input_name}-5": "loudness",
        f"{input_name}-6": "mode",
        f"{input_name}-7": "speechiness",
        f"{input_name}-8": "acousticness",
        f"{input_name}-9": "instrumentalness",
        f"{input_name}-10": "liveness",
        f"{input_name}-11": "valence",
        f"{input_name}-12": "tempo"
    },
    "outputMapping": {
    }
}

response = requests.post(url, headers=headers, json=payload)
print(response.text)

Now we can subscribe to our drift detection metric which will cause our TrustyAI Service to continously publish drift (specifically meanshift) metrics.  

The meanshift metric track how different the distribution of the training data looks like compared to the new data we send it.  

For each individual input and output feature we will get a "p-value" between 0 and 1. A p-value of 1.0 indicates a very high likelihood that the train and test data come from the same distribution, while a p-value < 0.05 indicates a statistically significant drift between train and test set.


In [ ]:
# Monitor meanshift
endpoint = "/metrics/drift/meanshift/request"
url = urljoin(base_url, endpoint)

payload = {
    "modelId": model_name,
    "referenceTag": "TRAINING"
}

response = requests.post(url, headers=headers, json=payload)
print(response.text)

To make sure all looks correct, we can ask the TrustyAI Service how our current setup looks like.  
We will get back information on what metrics we have subscribed to, as well as what data has been added.  

In [ ]:
# See what we have registered
endpoint = "/info"
url = urljoin(base_url, endpoint)
response = requests.get(url, headers=headers)
print(response.text)

### Send a request
Finally, we need to send a single request to our model server for TrustyAI to start publishing the metrics. This is so that it has at least one inference datapoint to compare its training data to.

In [ ]:
infer_url = f"{infer_endpoint}/v2/models/{model_name}/infer"

def rest_request(data):
    json_data = {
        "inputs": [
            {
                "name": input_name,
                "shape": [1, 13],
                "datatype": "FP32",
                "data": data
            }
        ]
    }

    response = requests.post(infer_url, json=json_data, verify=True)
    response_dict = response.json()
    return response_dict['outputs'][0]['data']

prediction = rest_request(data[0].tolist())

Additonally, we can also monitor the avearage value of any feature to see how it changes over time.

In [ ]:
# Monitor the average value
endpoint = "/metrics/identity/request"
url = urljoin(base_url, endpoint)

payload = {
    "modelId": model_name,
    "columnName": "duration_ms",
    "batchSize": 256,
}

response = requests.post(url, headers=headers, json=payload)
print(response.text)

Let's go to the next Notebook to create some drift and observe the metrics in OpenShift UI 👉 [jukebox/4-metrics/2-introducing_drift.ipynb](2-introducing_drift.ipynb)